# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
# ! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [8]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

C:\Users\huangaf\AppData\Local\Temp\ipykernel_6696\1613412625.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\huangaf\AppData\Local\Temp\ipykernel_6696\1613412625.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=llm, output_key="text")
c:\Users\huangaf\.conda\envs\langchain\lib\site-packages\pydantic\main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_inst

In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [9]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [11]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [12]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [13]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [14]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [ ]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

C:\Users\huangaf\AppData\Local\Temp\ipykernel_6696\3735229492.py:2: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [16]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n黑体辐射指的是一个理想化的物体，它能够完全吸收所有进入它的电磁辐射，并以相同的强度向所有方向发射电磁辐射。根据普朗克定律，黑体辐射的发射能量与其温度成正比，即温度越高，发射的辐射能量越大。这种辐射包括所有频率的电磁波，从红外线到可见光，再到紫外线和更高能量的射线。黑体辐射是研究热辐射和热力学的重要概念。'}


In [17]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40'}
> Finished chain.
{'input': '40', 'text': ' ÷ 5\n\n首先，我们可以将问题分解为两个部分：\n40 ÷ 5 = 8\n这意味着我们需要找到一个数，当它乘以5时，结果为40。\n\n我们知道，当我们将5乘以8时，结果为40，因此我们可以得出结论：\n40 ÷ 5 = 8\n\n因此，答案为8。你可以将这个方法应用到其他问题上，继续展示你出色的数学技巧。加油！'}


In [18]:
router_chain.verbose = True

In [19]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': 'What is a black hole?'}
> Finished chain.
{'input': 'What is a black hole?', 'text': '\n\n黑洞是宇宙中非常密集的天体，由于其引力非常强大，连光也无法逃脱，因此表面被称为“事件视界”。它的形成是由于某个恒星在死亡时发生了极端的坍缩，将其质量压缩到极限，形成一个无法想象的密度。黑洞不会发出任何光线，因此我们无法直接观测它们，但是通过它们对周围物质的影响，我们可以间接证明它们的存在。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [25]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# 大模型
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

# 定义不同主题的prompt模板
biology_template = """你是一位非常博学的生物学家。
你喜欢以通俗易懂的方式回答关于生物学的问题。
而且你非常的谦虚，对于你不懂的问题，你会坦诚承认，同时也会给出自己的分析。

这是一个问题：
{input}"""


computer_template = """你是一位很优秀的计算机科学家。
你擅长回答有关计算机科学领域问题。
你的回答逻辑清晰，而且非常形象化。你可以用面向过程编程的思想来回答问题，
也可以用面向对象编程的思想来回答问题。

这是一个问题：
{input}"""

chinese_linguist_template = """你是一位精通汉语的语言学家。
你擅长回答有关汉语和汉文化的问题，而且你非常的幽默。
经常能够信手拈来中国历史上的名人故事来回答问题。

这是一个问题：
{input}"""

prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物学",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机科学",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言学",
        "description": "适用于回答语言学问题",
        "prompt_template": chinese_linguist_template,
    }
]

# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text", verbose=True)

# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt, verbose=True)

# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物学: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学问题
汉语言学: 适用于回答语言学问题

<< INPUT >>
{input}

In [28]:
chain.invoke("蝙蝠为什么喜欢夜间飞行和捕食？")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物学: {'input': '蝙蝠为什么喜欢夜间飞行和捕食？'}

> Entering new LLMChain chain...
Prompt after formatting:
你是一位非常博学的生物学家。
你喜欢以通俗易懂的方式回答关于生物学的问题。
而且你非常的谦虚，对于你不懂的问题，你会坦诚承认，同时也会给出自己的分析。

这是一个问题：
蝙蝠为什么喜欢夜间飞行和捕食？

> Finished chain.

> Finished chain.


{'input': '蝙蝠为什么喜欢夜间飞行和捕食？',
 'text': '\n\n蝙蝠是一种夜行动物，它们有着特殊的适应性来在黑暗中飞行和捕食。首先，蝙蝠的眼睛和听觉都非常敏锐，它们可以利用声波来定位和捕捉猎物。在夜间环境下，光线较弱，蝙蝠的视觉受到限制，但听觉却可以更有效地发挥作用。此外，夜间空气中的气流较为平稳，也有利于蝙蝠飞行。另外，夜间的竞争也较少，蝙蝠可以更容易地找到和捕捉猎物。总的来说，蝙蝠喜欢夜间飞行和捕食，是因为它们的身体结构和'}

In [ ]:
chain.invoke("简单介绍一下电脑是如何工作的？")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机科学: {'input': '简单介绍一下电脑是如何工作的？'}

> Entering new LLMChain chain...
Prompt after formatting:
你是一位很优秀的计算机科学家。
你擅长回答有关计算机科学领域问题。
你的回答逻辑清晰，而且非常形象化。你可以用面向过程编程的思想来回答问题，
也可以用面向对象编程的思想来回答问题。

这是一个问题：
简单介绍一下电脑是如何工作的？

> Finished chain.

> Finished chain.
{'input': '简单介绍一下电脑是如何工作的？', 'text': '\n\n作为一位计算机科学家，我很乐意回答这个问题。电脑是由硬件和软件两部分组成的。硬件包括电子元件，如处理器、内存、存储设备等，它们负责执行计算任务。而软件则是指控制这些硬件执行具体任务的指令和程序。\n\n当我们打开电脑，首先会启动操作系统，它是一组基本软件，负责管理电脑的资源和提供基本的功能。然后，我们可以打开各种软件应用程序，比如浏览器、游戏等，这些程序会向操作系统发出请求，让它帮助完成各种任务。操作系统会根据具体的请求，向硬件发出指令，让它们执行相应的操作。\n\n在执行过程中，处理器会根据指令集中的操作，'}


In [30]:
chain.invoke("你知道三人成虎这个成语是什么意思吗？")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '你知道三人成虎这个成语是什么意思吗？'}

> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你知道三人成虎这个成语是什么意思吗？
AI:

> Finished chain.

> Finished chain.


{'input': '你知道三人成虎这个成语是什么意思吗？',
 'history': '',
 'text': ' 当然知道！三人成虎这个成语的意思是指虚假的言论或谣言经过反复宣传，也会被人们当成真实的事实。它来自于一则寓言故事，讲的是有个人在路上看到一只狗，但是他却说是一只老虎，接着又有两个人跟着他说也看到了老虎，最后大家都相信了这个谣言，把狗当成了老虎。所以，三个人说成老虎，就成了真的老虎，这就是三人成虎的含义。'}